# Анализ всех видео в директории


In [ ]:
import os
import torch
import cv2
import numpy as np
import sqlite3
import pandas as pd
from tqdm import tqdm
from PIL import Image
from pytorchvideo.models.hub import i3d_r50  # Правильная I3D модель для классификации видео
from torchvision import transforms
from torch.nn.functional import cosine_similarity
import pickle


# Подключение к SQLite базе данных
def create_db(db_name='videos.db'):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    
    # Создание таблицы для видео с добавлением колонки для признаков (features)
    c.execute('''
        CREATE TABLE IF NOT EXISTS video_data (
            created TEXT,
            uuid TEXT PRIMARY KEY,
            link TEXT,
            is_duplicate BOOLEAN,
            duplicate_for TEXT,
            features BLOB  -- колонка для хранения признаков в бинарном формате
        )
    ''')
    conn.commit()
    return conn, c

# Запись данных в SQLite (включая признаки)
def save_to_db(cursor, conn, row, features):
    # Сериализуем признаки с помощью pickle
    features_blob = pickle.dumps(features)
    
    cursor.execute('''
        INSERT OR REPLACE INTO video_data (created, uuid, link, is_duplicate, duplicate_for, features)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['created'], row['uuid'], row['link'], row['is_duplicate'], row['duplicate_for'], features_blob))
    
    conn.commit()

# Функция для загрузки данных из CSV
def load_data_from_csv(csv_file):
    return pd.read_csv(csv_file)

# Функция для извлечения кадров из видео
def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // num_frames)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    
    cap.release()
    return frames

# Преобразование для входных данных модели
def preprocess(frames, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    ])
    
    # Преобразуем каждый кадр из numpy в PIL.Image перед трансформацией
    frames = [transform(Image.fromarray(frame)) for frame in frames]
    
    # Преобразуем список тензоров в один тензор с нужными размерами и переносим на GPU
    frames = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).to(device)  # (B, C, T, H, W)
    return frames

# Функция для извлечения признаков видео с использованием I3D
def extract_features(model, video_path, device):
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return None  # если не удалось извлечь кадры
    input_tensor = preprocess(frames, device)
    with torch.no_grad():
        features = model(input_tensor)
    return features.squeeze(0).cpu()  # Переносим обратно на CPU для хранения

# Загрузка предобученной модели I3D
def load_i3d_model(device):
    model = i3d_r50(pretrained=True).eval().to(device)  # Переносим модель на GPU
    return model

# Функция для поиска дубликатов по косинусному сходству
def find_duplicates(features_dict, threshold=0.8):
    duplicates = []
    video_names = list(features_dict.keys())
    
    for i in range(len(video_names)):
        for j in range(i + 1, len(video_names)):
            vid1, vid2 = video_names[i], video_names[j]
            similarity = cosine_similarity(features_dict[vid1].unsqueeze(0), features_dict[vid2].unsqueeze(0))
            if similarity.item() > threshold:
                duplicates.append((vid1, vid2, similarity.item()))
    
    return duplicates

arr = None
# Основная функция для поиска дубликатов
def detect_duplicates_in_videos(csv_file, video_folder, device='cpu'):
    global arr
     # Создаем подключение к базе данных
    conn, cursor = create_db()

    # Загружаем данные из CSV
    data = load_data_from_csv(csv_file)[:]
    
    device = torch.device(device)
    model = load_i3d_model(device)
    
    # Извлечение признаков из всех видео с прогресс-баром
    features_dict = {}
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Обработка видео"):
        video_name = row['uuid']
        video_path = os.path.join(video_folder, video_name + '.mp4')
#         print(video_path)
        # Извлекаем признаки из видео
        features = extract_features(model, video_path, device)
        if features is not None:
            features_dict[row['uuid']] = features
#         print(features_dict)
    
    # Поиск дубликатов
    duplicates = find_duplicates(features_dict)

    # Добавляем результаты поиска дубликатов в данные и сохраняем в базу
    for idx, row in data.iterrows():
        row_data = {
            'created': row['created'],
            'uuid': row['uuid'],
            'link': row['link'],
            'is_duplicate': False,
            'duplicate_for': None
        }
        
        # Если дубликат найден
        for dup in duplicates:
            if row['uuid'] == dup[0]:  # Если это дубликат
                row_data['is_duplicate'] = True
                row_data['duplicate_for'] = dup[1]  # UUID оригинала
        
        # Сохраняем строку в базу данных, включая сериализованные признаки
        save_to_db(cursor, conn, row_data, features_dict[row['uuid']])
    
    # Закрываем соединение с базой данных
    conn.close()
    
    # Вывод результата
    if duplicates:
        print("Найдены дубликаты:")
        for dup in duplicates:
            print(f"{dup[0]} и {dup[1]} - Косинусное сходство: {dup[2]:.4f}")
    else:
        print("Дубликаты не найдены.")
        
    arr = duplicates

# Укажите путь к папке с видео
csv_file_path = r'F:\hac\test\test.csv'
video_folder_path = r'F:\hac\test\test_dataset'
detect_duplicates_in_videos(csv_file_path, video_folder_path, device='cuda' if torch.cuda.is_available() else 'cpu')

# Добавление одного видео

In [ ]:
import os
import torch
import cv2
import numpy as np
import sqlite3
import pickle
from PIL import Image
from pytorchvideo.models.hub import i3d_r50
from torchvision import transforms
from torch.nn.functional import cosine_similarity

# Подключение к SQLite базе данных
def connect_db(db_name='videos.db'):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    return conn, c

# Преобразование для входных данных модели
def preprocess(frames, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
    ])
    
    frames = [transform(Image.fromarray(frame)) for frame in frames]
    frames = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).to(device)  # (B, C, T, H, W)
    return frames

# Функция для извлечения кадров из видео
def extract_frames(video_path, num_frames=32):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // num_frames)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    
    cap.release()
    return frames

# Функция для извлечения признаков видео с использованием I3D
def extract_features(model, video_path, device):
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return None  # если не удалось извлечь кадры
    input_tensor = preprocess(frames, device)
    with torch.no_grad():
        features = model(input_tensor)
    return features.squeeze(0).cpu()  # Переносим обратно на CPU для хранения

# Функция для загрузки признаков из БД
def load_features_from_db(cursor):
    cursor.execute("SELECT uuid, features FROM video_data")
    rows = cursor.fetchall()
    features_dict = {}
    
    # Десериализация признаков
    for row in rows:
        uuid, features_blob = row
        features = pickle.loads(features_blob)
        features_dict[uuid] = features
    
    return features_dict

# Сравнение признаков нового видео с признаками в базе данных
def find_duplicate_in_db(new_features, features_dict, threshold=0.9):
    for uuid, features in features_dict.items():
        similarity = cosine_similarity(new_features.unsqueeze(0), features.unsqueeze(0))
        if similarity.item() > threshold:
            return uuid  # Возвращаем UUID дубликата, если найден
    
    return None

# Функция для добавления нового видео в БД
def save_new_video_to_db(cursor, conn, row, features):
    features_blob = pickle.dumps(features)  # Сериализация признаков
    cursor.execute('''
        INSERT OR REPLACE INTO video_data (created, uuid, link, is_duplicate, duplicate_for, features)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['created'], row['uuid'], row['link'], row['is_duplicate'], row['duplicate_for'], features_blob))
    conn.commit()

# Функция для обработки нового видео
def process_new_video(video_path, created, uuid, link, device='cpu', db_name='videos.db'):
    conn, cursor = connect_db(db_name)
    
    # Загрузка модели
    device = torch.device(device)
    model = i3d_r50(pretrained=True).eval().to(device)
    
    # Извлекаем признаки нового видео
    new_features = extract_features(model, video_path, device)
    if new_features is None:
        print(f"Не удалось извлечь признаки для видео: {video_path}")
        return
    
    # Загружаем признаки всех видео из базы данных
    features_dict = load_features_from_db(cursor)
    
    # Поиск дубликата
    duplicate_uuid = find_duplicate_in_db(new_features, features_dict)
    
    # Подготовка записи для нового видео
    row_data = {
        'created': created,
        'uuid': uuid,
        'link': link,
        'is_duplicate': duplicate_uuid is not None,
        'duplicate_for': duplicate_uuid
    }
    
    # Сохранение нового видео (как дубликата или нет) в базу данных
    save_new_video_to_db(cursor, conn, row_data, new_features)
    
    # Закрываем соединение с базой данных
    conn.close()
    
    if duplicate_uuid:
        print(f"Найден дубликат! Видео {uuid} является дубликатом видео {duplicate_uuid}.")
    else:
        print(f"Видео {uuid} уникально и добавлено в базу данных.")

# Пример использования
new_video_path = r'F:\hac\test\test_dataset\337fdbe6-2bc7-4bc7-931e-d94ada927732.mp4'
new_created = '2024-09-28 12:00:00'
new_uuid = '337fdbe6-2bc7-4bc7-931e-d94ada927732'
new_link = 'new-video-link.mp4'

# Запуск обработки нового видео
process_new_video(new_video_path, new_created, new_uuid, new_link, device='cuda' if torch.cuda.is_available() else 'cpu')

# Просмотр видео

In [ ]:
import cv2
import os


# Пример массива с парами путей до видеофайлов
video_pairs = [['337fdbe6-2bc7-4bc7-931e-d94ada927732', '337fdbe6-2bc7-4bc7-931e-d94ada927ede']]

# Функция для показа двух видео одновременно
def show_video_pair(video_pair):
#     os.path.join(r'F:\hac\train\test', video_pair[0])
    # Открываем видеофайлы
    cap1 = cv2.VideoCapture(os.path.join(r'F:\hac\test\test_dataset', video_pair[0] + '.mp4'))
    cap2 = cv2.VideoCapture(os.path.join(r'F:\hac\test\test_dataset', video_pair[1] + '.mp4'))
    
    while cap1.isOpened() and cap2.isOpened():
        # Читаем кадры из видео
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        
        # Если кадры не были считаны, выходим из цикла
        if not ret1 or not ret2:
            break
        
        # Масштабируем кадры, если нужно, чтобы они были одинакового размера
        frame1 = cv2.resize(frame1, (360, 640))  # Измените размеры, если необходимо
        frame2 = cv2.resize(frame2, (360, 640))  # Измените размеры, если необходимо
        
        # Объединяем два видео горизонтально для показа на одном экране
        combined_frame = cv2.hconcat([frame1, frame2])
        
        # Показываем объединенное видео
        cv2.imshow('Videos', combined_frame)
        
        # Ожидание нажатия клавиши
        key = cv2.waitKey(30)  # Задержка в 30 мс между кадрами
        
        # Если нажата клавиша 'n', переходим к следующей паре видео
        if key == ord('n'):
            break
        
        # Если нажата клавиша 'p', возвращаемся к предыдущей паре видео
        if key == ord('p'):
            cap1.release()
            cap2.release()
            return 'prev'
    
    # Освобождаем ресурсы
    cap1.release()
    cap2.release()
    
    return 'next'

# Основная логика программы
def main(video_pairs):
    current_pair_index = 0
    
    while 0 <= current_pair_index < len(video_pairs):
        # Показываем текущую пару видео
        action = show_video_pair(video_pairs[current_pair_index])
        
        # Обработка нажатий клавиш
        if action == 'next':
            current_pair_index += 1  # Переход к следующей паре
        elif action == 'prev' and current_pair_index > 0:
            current_pair_index -= 1  # Возврат к предыдущей паре
    
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(video_pairs)
